# Linelist generator

This is a simple notebook to generate a linelist for use by `cloudy`.

In [29]:
import numpy as np
from synthesizer.photoionisation import cloudy23

In [30]:

wavelength_type = 'standard'
# wavelength_type = 'vacuum'

# the reference line to use
if wavelength_type == 'standard':
    reference_line = 'H 1 4861.32A'
if wavelength_type == 'vacuum':
    reference_line = 'H 1 4862.69A'

# log constrast relative to Hbeta to consider
contrast_stars = 1.5
contrast_agn = 1.0

# minimum wavelength of lines to consider
min_wavelength = 1000

# maximum wavelength of lines to consider
max_wavelength = 25000

### Stellar models

In [31]:

linelist = []

model = r'metallicity_'+wavelength_type
for i in range(4):
    i += 1
    line_ids, blends, wavelengths, intrinsic, emergent = cloudy23.read_lines(rf'{model}/{i}')
    Hbeta = emergent[line_ids==reference_line][0]
    s = (emergent > (Hbeta - contrast_stars)) & (wavelengths<max_wavelength) & (wavelengths>min_wavelength) & (blends == False)
    print(i, np.sum(s))
    linelist += list(line_ids[s])



1 78
2 62
3 34
4 24


### AGN models

In [32]:

model = r'relagn_'+wavelength_type

for i in range(3):
    i += 1
    line_ids, blends, wavelengths, intrinsic, emergent = cloudy23.read_lines(rf'{model}/{i}')
    Hbeta = emergent[line_ids==reference_line][0]
    s = (emergent > (Hbeta - contrast_agn)) & (wavelengths<max_wavelength) & (wavelengths>min_wavelength) & (blends == False)
    print(i, np.sum(s))
    linelist += list(line_ids[s])


1 224
2 118
3 49


In [33]:
linelist = list(set(linelist))
linelist.sort()

# print(linelist)
print(len(linelist))

linelist_ = []

for line in linelist:

    element, ion, wavelength = line.split(' ')

    if len(ion)==1:

        line_ = f'{element} {ion} {wavelength}'
        print(line_)
        linelist_.append(line_)


with open(f'linelist-{wavelength_type}.dat', 'w') as file:
    file.writelines('\n'.join(linelist_) + '\n')

252
Al 2 1670.79A
Ar 3 7135.79A
Ar 3 7751.11A
Ar 4 2853.66A
C 1 1657.91A
C 1 1992.01A
C 1 2582.90A
C 2 1334.53A
C 2 1335.66A
C 2 1335.71A
C 2 2325.40A
C 2 2326.93A
C 3 1906.68A
C 3 1908.73A
C 4 1548.19A
C 4 1550.77A
Ca 2 7291.47A
Ca 2 7323.89A
Cl 2 8578.70A
Fe 2 1.25668m
Fe 2 1.27877m
Fe 2 1.29427m
Fe 2 1.32055m
Fe 2 1.32777m
Fe 2 1.37181m
Fe 2 1.53348m
Fe 2 1.59948m
Fe 2 1.64355m
Fe 2 1.66377m
Fe 2 1.67688m
Fe 2 1.71113m
Fe 2 1.74494m
Fe 2 1.79711m
Fe 2 1.80002m
Fe 2 1.80940m
Fe 2 1.89541m
Fe 2 1.95361m
Fe 2 2395.63A
Fe 2 2399.24A
Fe 2 2406.66A
Fe 2 2410.52A
Fe 2 2598.37A
Fe 2 2607.09A
Fe 2 2611.87A
Fe 2 2613.82A
Fe 2 2625.67A
Fe 2 2628.29A
Fe 2 2631.05A
Fe 2 2631.32A
Fe 2 4243.97A
Fe 2 4276.84A
Fe 2 4287.39A
Fe 2 4319.62A
Fe 2 4346.86A
Fe 2 4352.79A
Fe 2 4358.37A
Fe 2 4359.33A
Fe 2 4413.78A
Fe 2 4416.27A
Fe 2 4452.10A
Fe 2 4474.90A
Fe 2 4814.54A
Fe 2 4874.50A
Fe 2 4889.62A
Fe 2 4905.35A
Fe 2 4923.92A
Fe 2 4947.39A
Fe 2 4973.40A
Fe 2 5005.52A
Fe 2 5018.44A
Fe 2 5020.25A
Fe 2 5049.30A
